Where are the delivery ‘dead zones’—areas with minimal restaurant coverage?

In [ ]:
import os

os.chdir('C:/Users/esram/OneDrive/Desktop/esrabecode/delivery-market-analysis')

os.getcwd()

'C:\\Users\\esram\\OneDrive\\Desktop\\esrabecode\\delivery-market-analysis'

SIMPLE

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
conn = sqlite3.connect('data/takeaway.db')

query= """
SELECT l.postalCode,
    COUNT(r.primarySlug) AS restaurant_count
    
FROM restaurants r

JOIN locations_to_restaurants ltr 
ON r.primarySlug = ltr.restaurant_id

JOIN locations l 
ON l.id = ltr.location_id
GROUP BY l.postalCode
HAVING COUNT(r.primarySlug) < 10

ORDER BY restaurant_count ASC;
"""

top_10_deadzone = pd.read_sql_query(query, conn)
top_10_deadzone

FINAL TABLE WITH LOCATIONS AND NAMES

In [18]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/takeaway.db')

query = """

WITH dead_zones AS (
    SELECT l.postalCode,
        COUNT(r.primarySlug) AS restaurant_count
    FROM restaurants r
    JOIN locations_to_restaurants ltr
        ON r.primarySlug = ltr.restaurant_id
    JOIN locations l
        ON l.id = ltr.location_id
    GROUP BY l.postalCode
    HAVING COUNT(r.primarySlug) < 10
)


SELECT r.primarySlug,
       r.name,
       r.ratings,
       r.ratingsNumber,
       l.postalCode,
       l.latitude,
       l.longitude,
       dz.restaurant_count
FROM restaurants r
JOIN locations_to_restaurants ltr
    ON r.primarySlug = ltr.restaurant_id
JOIN locations l
    ON l.id = ltr.location_id
JOIN dead_zones dz
    ON l.postalCode = dz.postalCode

ORDER BY dz.restaurant_count ASC, l.postalCode ASC;
"""

deadzone_df = pd.read_sql_query(query, conn)
deadzone_df


,primarySlug,name,ratings,ratingsNumber,postalCode,latitude,longitude,restaurant_count
0,tucker-food,Tucker Food,3.9,171,8951,50.766352,2.783481,2
1,the-pizza-valley-dranouter-8951,The Pizza Valley,4.7,16,8951,50.766352,2.783481,2
2,cleopatra-poperinge,Cleopatra,3.6,111,8978,50.859092,2.664685,2
3,cleopatra-poperinge,Cleopatra,3.6,111,8978,50.859203,2.620179,2
4,down-town,Down-Town,3.6,110,9981,51.282329,3.546151,2
...,...,...,...,...,...,...,...,...
300,pitta-pizza-izmir-eeklo,Pitta & Pizza Izmir Eeklo,4.7,547,9991,51.203003,3.485724,9
301,pizza-midas-1,Pizza Midas,4.4,503,9991,51.203003,3.485724,9
302,pizza-hut-delivery-maldegem,Pizza Hut Delivery,3.6,256,9991,51.203003,3.485724,9
303,goodys,Goody's,4.5,2219,9991,51.203003,3.485724,9


In [19]:
import folium
from folium.plugins import MarkerCluster

map_center = [deadzone_df['latitude'].mean(), deadzone_df['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

for idx, row in deadzone_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5 + row['restaurant_count'], 
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=f"PostalCode: {row['postalCode']}<br>"
              f"Restaurants: {row['restaurant_count']}<br>"
              f"Example: {row['name']}"
    ).add_to(marker_cluster)

m.save("dead_zones_map.html")
